# マルチタスク強化学習のサンプル効率

参考：
* [Sample Complexity of Multi-task Reinforcement Learning](https://arxiv.org/abs/1309.6821)：もっと良い論文ありそう．問題設定と解き方は参考になる．

強化学習でマルチタスクを扱う場合，しばしば転移学習が用いられますが，なぜそれが良いのか？は自明ではありません．
強化学習におけるサンプル効率の理論を見てみましょう．

表記：
* ホライゾン$H$
* タスクの数$T$
    * MDPの集合$\mathcal{M}$．$|\mathcal{M}|=C$とします．
    * それぞれのタスクは$\mathcal{M}$からサンプルされます
    * タスクは状態行動空間は同じですが，報酬や遷移が異なります．[文脈付きMDP](RL_multi_task_contextual_MDP.ipynb)に似てます


## マルチタスク強化学習アルゴリズム

次の手順でマルチタスクMDPを解きます

1. 入力：サンプルするタスクの数$T$と分類するMDPの数$\bar{C}$
2. for $t=1,2, \ldots, T_1$ do
    1. $M_t \in \mathcal{M}$をサンプル
    2. $M_t$において，$H$ステップだけ$E^3$アルゴリズムを走らせ，$o\left(s, a, s^{\prime}, t\right)$のカウントを得ます．$E^3$アルゴリズムについては[Near-Optimal Reinforcement Learning in Polynomial Time](https://www.cis.upenn.edu/~mkearns/papers/KearnsSinghE3.pdf)参照．
3. 集めたカウントを利用して推定したそれぞれの遷移$\hat{P}_t$について，$\|\hat{P}_{t_1} - \hat{P}_{t_2}\|_\infty \leq \varepsilon$であるような$t_1$と$t_2$は同じ分類とします．
4. for $t=T_1 + 1, \ldots, T$ do
    1. 未知の$M_t \in \mathcal{M}$を受け取る
    2. この$M_t$が既知であるなら，...（この後がよくわからない．読みづらい...）